# World Bank - Education Attainment Summarizing
Reduced '.csv' files from WB_data Notebook

In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import requests
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np
import time


# Impor the OpenWeatherMap API key
from api_keys import geoapify_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [2]:
# WB data set
WB_africadata_2000_bins = Path("Resources\WB_africadata_2000_bins.csv")

In [3]:
# Read data file with the Pandas library
# encoding?, i.e encoding="ISO-8859-1"
WB_africadata_2000_df2 = pd.read_csv(WB_africadata_2000_bins)

In [4]:
WB_africadata_2000_df2

,series_id,country_code,country_name,year,value,Year Group
0,SH.HIV.INCD.TL,KEN,Kenya,2017,3.400000e+04,2016 - 2020
1,SH.HIV.INCD.TL,LBR,Liberia,2017,1.600000e+03,2016 - 2020
2,SH.HIV.INCD.TL,LSO,Lesotho,2017,1.000000e+04,2016 - 2020
3,SH.HIV.INCD.TL,MDG,Madagascar,2017,7.200000e+03,2016 - 2020
4,SH.HIV.INCD.TL,MOZ,Mozambique,2017,1.300000e+05,2016 - 2020
...,...,...,...,...,...,...
652157,VA.NO.SRC,WSM,Samoa,2006,3.000000e+00,2006 - 2010
652158,FM.LBL.BMNY.GD.ZS,TCD,Chad,2010,1.146678e+01,2006 - 2010
652159,NE.CON.PRVT.CN.AD,SDN,Sudan,2000,2.199464e+10,2000 - 2005
652160,NE.CON.PRVT.CN.AD,SDN,Sudan,2003,3.190503e+10,2000 - 2005


# Slicing of the Data
###In order to analyze the data and to understand the impact of the Foreign Aid with regards to poverty rate, literacy and mortality, the data was sliced by the "series_id". The deifinition of the diferent "series_id" values (indicators) is described in a second file downloaded from the Nasdaq API and saved as 'WB_metadata_df'.

###To facilitate/expedite the search and filtering of the relevant a function was created to search the text strings : "key_word". This function is case sensitive and it's used is combined with the review of the search output selection to identify the indicators that best match our work objective.

In [5]:
# WB id series definition data set
WB_metadata = Path("Resources/WB_METADATA_f7ce7fba293ccc6eb39cdf15fb097982.csv")

In [6]:
# Read data file with the Pandas library
WB_metadata_df = pd.read_csv(WB_metadata)

In [7]:
WB_metadata_df

,series_id,name,description
0,DC.DAC.DEUL.CD,"Net bilateral aid flows from DAC donors, Germa...",Net bilateral aid flows from DAC donors are th...
1,RQ.STD.ERR,Regulatory Quality: Standard Error,Regulatory Quality captures perceptions of the...
2,EG.USE.PCAP.KG.OE,Energy use (kg of oil equivalent per capita),Energy use refers to use of primary energy bef...
3,EN.POP.EL5M.UR.ZS,Urban population living in areas where elevati...,Urban population below 5m is the percentage of...
4,per_lm_alllm.cov_q1_tot,Coverage of unemployment benefits and ALMP in ...,Coverage of unemployment benefits and active l...
...,...,...,...
1479,SL.TLF.BASC.FE.ZS,"Labor force with basic education, female (% of...",The ratio of the labor force with basic educat...
1480,SL.GDP.PCAP.EM.KD,GDP per person employed (constant 2021 PPP $),GDP per person employed is gross domestic prod...
1481,SL.TLF.INTM.MA.ZS,"Labor force with intermediate education, male ...",The ratio of the labor force with intermediate...
1482,SL.UEM.NEET.FE.ZS,"Share of youth not in education, employment or...","Share of youth not in education, employment or..."


In [8]:
# This function will search within a text string for a given 'key word'. It is case sensitive

def key_word(df, search_col, word_txt, result_col):
    i=0   
    result_ls = []
    for item in df[search_col]:
        text_ls = [x.strip() for x in df[search_col][i].split()]
        if word_txt in text_ls:
            result_ls.append(df[result_col][i])
        i+=1
    if i >= df[search_col].count():
        if len(result_ls) > 0:
            return result_ls
        else:
            return print("Keyword Not found")

In [9]:
education_search = key_word(WB_metadata_df, 'name', 'Educational', 'series_id')

In [10]:
len(education_search)

24

In [11]:
education_search2= WB_metadata_df[WB_metadata_df['series_id'].isin(education_search)]

In [12]:
education_search2

,series_id,name,description
402,SE.SEC.CUAT.LO.ZS,"Educational attainment, at least completed low...",The percentage of population ages 25 and over ...
515,SE.SEC.CUAT.PO.FE.ZS,"Educational attainment, at least completed pos...",The percentage of population ages 25 and over ...
518,SE.TER.CUAT.ST.MA.ZS,"Educational attainment, at least completed sho...",The percentage of population ages 25 and over ...
556,SE.SEC.CUAT.UP.ZS,"Educational attainment, at least completed upp...",The percentage of population ages 25 and over ...
576,SE.TER.CUAT.MS.ZS,"Educational attainment, at least Master's or e...",The percentage of population ages 25 and over ...
632,SE.TER.CUAT.MS.FE.ZS,"Educational attainment, at least Master's or e...",The percentage of population ages 25 and over ...
640,SE.SEC.CUAT.LO.FE.ZS,"Educational attainment, at least completed low...",The percentage of population ages 25 and over ...
662,SE.TER.CUAT.ST.ZS,"Educational attainment, at least completed sho...",The percentage of population ages 25 and over ...
677,SE.TER.CUAT.MS.MA.ZS,"Educational attainment, at least Master's or e...",The percentage of population ages 25 and over ...
685,SE.SEC.CUAT.UP.FE.ZS,"Educational attainment, at least completed upp...",The percentage of population ages 25 and over ...


# Selection of Indicators for Analysis

###Indicators are selected out of the search result and used to create redu ed data sets. In the case of the Mortality indicator, we selected three indicators : mortality under age 5-females only, mortality under age 5-males only, and mortality under age 5-combined. Values listed are the rate per 100,000 parts.

###In order to organize the data, we prepared 6 data sets. One for each indicator.

In [13]:
# Literacy indicators : percentage of population
#SE.SEC.CUAT.LO.ZS - The percentage of population ages 25 and over that attained or completed lower secondary education.
#SE.SEC.CUAT.UP.ZS - The percentage of population ages 25 and over that attained or completed upper secondary education.
#SE.TER.CUAT.MS.ZS - The percentage of population ages 25 and over that attained or completed Master's or equivalent.

In [14]:
WB_education_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'].isin(["SE.SEC.CUAT.LO.ZS", "SE.SEC.CUAT.UP.ZS", "SE.TER.CUAT.MS.ZS" ])]
WB_education_df

,series_id,country_code,country_name,year,value,Year Group
60104,SE.SEC.CUAT.UP.ZS,WSM,Samoa,2001,63.037628,2000 - 2005
60105,SE.SEC.CUAT.UP.ZS,TZA,Tanzania,2002,1.622880,2000 - 2005
60106,SE.SEC.CUAT.UP.ZS,UGA,Uganda,2002,6.453940,2000 - 2005
60107,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2002,10.357510,2000 - 2005
60108,SE.SEC.CUAT.UP.ZS,SLE,Sierra Leone,2003,4.066010,2000 - 2005
...,...,...,...,...,...,...
324895,SE.SEC.CUAT.LO.ZS,NGA,Nigeria,2022,69.982292,2021 - 2023
324896,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2022,17.950159,2021 - 2023
324897,SE.SEC.CUAT.LO.ZS,SOM,Somalia,2022,11.111570,2021 - 2023
324898,SE.SEC.CUAT.LO.ZS,ZMB,Zambia,2022,65.410141,2021 - 2023


In [15]:
education_counts = WB_education_df['country_name'].value_counts()

print("Counts of entries for each country name:")
print(education_counts)

Counts of entries for each country name:
Rwanda                      28
Senegal                     23
Angola                      20
Tanzania                    18
Zambia                      18
Uganda                      17
Zimbabwe                    17
Nigeria                     16
Sierra Leone                15
Burkina Faso                15
Ghana                       15
Mauritania                  14
Togo                        14
Kenya                       14
Malawi                      14
Mozambique                  13
Liberia                     12
Botswana                    12
Ethiopia                    11
Egypt, Arab Rep.            11
Tunisia                     10
Chad                         9
Lesotho                      8
Madagascar                   8
Samoa                        4
Djibouti                     3
Sao Tome and Principe        3
Sudan                        3
Central African Republic     2
Somalia                      2
Name: country_name, dtype: in

## Educational Attainment : Lower Secondary (% Pop.> 25yrs)

In [16]:
WB_Educ_LS_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SE.SEC.CUAT.LO.ZS"]
WB_Educ_LS_df = WB_Educ_LS_df.rename(columns = {'value': 'Lower Secondary Educ. (% Pop.> 25yrs)'})
WB_Educ_LS_df

,series_id,country_code,country_name,year,Lower Secondary Educ. (% Pop.> 25yrs),Year Group
133129,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2010,12.582890,2006 - 2010
324759,SE.SEC.CUAT.LO.ZS,TZA,Tanzania,2002,5.986470,2000 - 2005
324760,SE.SEC.CUAT.LO.ZS,UGA,Uganda,2002,11.518880,2000 - 2005
324761,SE.SEC.CUAT.LO.ZS,ZWE,Zimbabwe,2002,46.016670,2000 - 2005
324762,SE.SEC.CUAT.LO.ZS,SLE,Sierra Leone,2003,16.482738,2000 - 2005
...,...,...,...,...,...,...
324894,SE.SEC.CUAT.LO.ZS,KEN,Kenya,2022,64.952110,2021 - 2023
324895,SE.SEC.CUAT.LO.ZS,NGA,Nigeria,2022,69.982292,2021 - 2023
324896,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2022,17.950159,2021 - 2023
324897,SE.SEC.CUAT.LO.ZS,SOM,Somalia,2022,11.111570,2021 - 2023


In [17]:
# Split up our data into groups based upon 'country_name' and 'year'
WB_Educ_LS_df_groups = WB_Educ_LS_df.groupby(['year', 'country_name'])
WB_Educ_LS_df_groups.head()

,series_id,country_code,country_name,year,Lower Secondary Educ. (% Pop.> 25yrs),Year Group
133129,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2010,12.582890,2006 - 2010
324759,SE.SEC.CUAT.LO.ZS,TZA,Tanzania,2002,5.986470,2000 - 2005
324760,SE.SEC.CUAT.LO.ZS,UGA,Uganda,2002,11.518880,2000 - 2005
324761,SE.SEC.CUAT.LO.ZS,ZWE,Zimbabwe,2002,46.016670,2000 - 2005
324762,SE.SEC.CUAT.LO.ZS,SLE,Sierra Leone,2003,16.482738,2000 - 2005
...,...,...,...,...,...,...
324894,SE.SEC.CUAT.LO.ZS,KEN,Kenya,2022,64.952110,2021 - 2023
324895,SE.SEC.CUAT.LO.ZS,NGA,Nigeria,2022,69.982292,2021 - 2023
324896,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2022,17.950159,2021 - 2023
324897,SE.SEC.CUAT.LO.ZS,SOM,Somalia,2022,11.111570,2021 - 2023


In [18]:
avg_Educ_LS_all = WB_Educ_LS_df_groups[['Lower Secondary Educ. (% Pop.> 25yrs)']].mean()
avg_Educ_LS_all

Lower Secondary Educ. (% Pop.> 25yrs)
year country_name                                       
2002 Tanzania                                   5.986470
     Uganda                                    11.518880
     Zimbabwe                                  46.016670
2003 Sierra Leone                              16.482738
2004 Angola                                    22.165590
...                                                  ...
2022 Kenya                                     64.952110
     Nigeria                                   69.982292
     Rwanda                                    17.950159
     Somalia                                   11.111570
     Zambia                                    65.410141

[141 rows x 1 columns]

In [19]:
avg_Educ_LS_yr = avg_Educ_LS_all.reset_index()

In [20]:
Educ_LS_pivot1 = avg_Educ_LS_yr.pivot(index='country_name', columns='year', values='Lower Secondary Educ. (% Pop.> 25yrs)')
Educ_LS_pivot1['Mean Mortality (2000 - 2023)'] = Educ_LS_pivot1.mean(axis=1)
Educ_LS_pivot1

year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Mean Mortality (2000 - 2023)
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,NaN,NaN,22.16559,NaN,NaN,NaN,NaN,10.561316,NaN,22.868460,...,28.923470,27.969761,NaN,NaN,NaN,38.469933,NaN,39.977886,NaN,27.276631
Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,67.608025,68.738220,71.157555,70.004646,69.377111
Burkina Faso,NaN,NaN,NaN,NaN,7.938160,2.03617,NaN,NaN,NaN,NaN,...,8.488350,NaN,NaN,NaN,12.670800,14.001460,NaN,11.453410,NaN,9.431392
Central African Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.548750,NaN,NaN,NaN,12.548750
Chad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.044390,NaN,NaN,...,NaN,NaN,NaN,NaN,13.263819,8.839040,NaN,NaN,NaN,10.715749
Djibouti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,23.784451,NaN,NaN,NaN,NaN,NaN,23.784451
"Egypt, Arab Rep.",NaN,NaN,NaN,NaN,44.430729,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,72.959137,NaN,NaN,52.602589,51.787548,NaN,55.445001
Ethiopia,NaN,NaN,NaN,NaN,NaN,7.21509,NaN,NaN,NaN,12.511690,...,NaN,NaN,11.082730,NaN,NaN,10.343800,NaN,NaN,NaN,11.762390
Ghana,NaN,NaN,NaN,NaN,40.595158,NaN,NaN,NaN,54.319618,NaN,...,NaN,51.817280,NaN,46.434349,NaN,NaN,NaN,NaN,NaN,48.816200


In [21]:
Educ_LS_pivot1['Mean Mortality (2000 - 2023)'] = Educ_LS_pivot1.mean(axis=1)
Educ_LS_pivot1

year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Mean Mortality (2000 - 2023)
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,NaN,NaN,22.16559,NaN,NaN,NaN,NaN,10.561316,NaN,22.868460,...,28.923470,27.969761,NaN,NaN,NaN,38.469933,NaN,39.977886,NaN,27.276631
Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,67.608025,68.738220,71.157555,70.004646,69.377111
Burkina Faso,NaN,NaN,NaN,NaN,7.938160,2.03617,NaN,NaN,NaN,NaN,...,8.488350,NaN,NaN,NaN,12.670800,14.001460,NaN,11.453410,NaN,9.431392
Central African Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,12.548750,NaN,NaN,NaN,12.548750
Chad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.044390,NaN,NaN,...,NaN,NaN,NaN,NaN,13.263819,8.839040,NaN,NaN,NaN,10.715749
Djibouti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,23.784451,NaN,NaN,NaN,NaN,NaN,23.784451
"Egypt, Arab Rep.",NaN,NaN,NaN,NaN,44.430729,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,72.959137,NaN,NaN,52.602589,51.787548,NaN,55.445001
Ethiopia,NaN,NaN,NaN,NaN,NaN,7.21509,NaN,NaN,NaN,12.511690,...,NaN,NaN,11.082730,NaN,NaN,10.343800,NaN,NaN,NaN,11.762390
Ghana,NaN,NaN,NaN,NaN,40.595158,NaN,NaN,NaN,54.319618,NaN,...,NaN,51.817280,NaN,46.434349,NaN,NaN,NaN,NaN,NaN,48.816200


In [22]:
# Split up our data into groups based upon 'country_name' and 'Year Group'
WB_Educ_LS_groups2 = WB_Educ_LS_df.groupby(['Year Group','country_name'])
WB_Educ_LS_groups2.head()

,series_id,country_code,country_name,year,Lower Secondary Educ. (% Pop.> 25yrs),Year Group
133129,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2010,12.582890,2006 - 2010
324759,SE.SEC.CUAT.LO.ZS,TZA,Tanzania,2002,5.986470,2000 - 2005
324760,SE.SEC.CUAT.LO.ZS,UGA,Uganda,2002,11.518880,2000 - 2005
324761,SE.SEC.CUAT.LO.ZS,ZWE,Zimbabwe,2002,46.016670,2000 - 2005
324762,SE.SEC.CUAT.LO.ZS,SLE,Sierra Leone,2003,16.482738,2000 - 2005
...,...,...,...,...,...,...
324894,SE.SEC.CUAT.LO.ZS,KEN,Kenya,2022,64.952110,2021 - 2023
324895,SE.SEC.CUAT.LO.ZS,NGA,Nigeria,2022,69.982292,2021 - 2023
324896,SE.SEC.CUAT.LO.ZS,RWA,Rwanda,2022,17.950159,2021 - 2023
324897,SE.SEC.CUAT.LO.ZS,SOM,Somalia,2022,11.111570,2021 - 2023


In [23]:
avg_Educ_LS_country = WB_Educ_LS_groups2[['Lower Secondary Educ. (% Pop.> 25yrs)']].mean()
avg_Educ_LS_country

Lower Secondary Educ. (% Pop.> 25yrs)
Year Group  country_name                                       
2000 - 2005 Angola                                    22.165590
            Malawi                                    14.237399
            Sierra Leone                              17.709894
            Tanzania                                   5.986470
            Uganda                                    11.518880
...                                                         ...
2021 - 2023 Rwanda                                    20.598156
            Somalia                                   11.111570
            Uganda                                    39.458057
            Zambia                                    64.654213
            Zimbabwe                                  89.623985

[81 rows x 1 columns]

In [24]:
avg_Educ_LS_bin = avg_Educ_LS_country.reset_index()

In [25]:
Educ_LS_pivot2 = avg_Educ_LS_bin.pivot(index='country_name', columns='Year Group', values='Lower Secondary Educ. (% Pop.> 25yrs)')
Educ_LS_pivot2['Mean Mortality (2000 - 2023)'] = Educ_LS_pivot2.mean(axis=1)
Educ_LS_pivot2

Year Group,2000 - 2005,2006 - 2010,2011 - 2015,2016 - 2020,2021 - 2023,Mean Mortality (2000 - 2023)
country_name,,,,,,
Angola,22.165590,10.561316,26.587230,38.469933,39.977886,27.552391
Botswana,NaN,NaN,NaN,68.173122,70.581100,69.377111
Burkina Faso,NaN,4.987165,8.488350,13.336130,11.453410,9.566264
Central African Republic,NaN,NaN,NaN,12.548750,NaN,12.548750
Chad,NaN,10.044390,NaN,11.051429,NaN,10.547909
Djibouti,NaN,NaN,NaN,23.784451,NaN,23.784451
"Egypt, Arab Rep.",NaN,44.430729,NaN,62.780863,51.787548,52.999713
Ethiopia,NaN,7.215090,15.085165,10.713265,NaN,11.004507
Ghana,NaN,47.457388,51.365938,46.434349,NaN,48.419225


## Educational Attainment : Upper Secondary (% Pop.> 25yrs)

In [26]:
WB_Educ_US_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SE.SEC.CUAT.UP.ZS"]
WB_Educ_US_df

,series_id,country_code,country_name,year,value,Year Group
60104,SE.SEC.CUAT.UP.ZS,WSM,Samoa,2001,63.037628,2000 - 2005
60105,SE.SEC.CUAT.UP.ZS,TZA,Tanzania,2002,1.622880,2000 - 2005
60106,SE.SEC.CUAT.UP.ZS,UGA,Uganda,2002,6.453940,2000 - 2005
60107,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2002,10.357510,2000 - 2005
60108,SE.SEC.CUAT.UP.ZS,SLE,Sierra Leone,2003,4.066010,2000 - 2005
...,...,...,...,...,...,...
149288,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2017,12.264890,2016 - 2020
149290,SE.SEC.CUAT.UP.ZS,KEN,Kenya,2019,33.388390,2016 - 2020
149292,SE.SEC.CUAT.UP.ZS,ZMB,Zambia,2021,36.321079,2021 - 2023
149293,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2021,64.132111,2021 - 2023


In [27]:
WB_Educ_US_df = WB_Educ_US_df.rename(columns = {'value': 'Upper Secondary Educ. (% Pop.> 25yrs)'})
WB_Educ_US_df

,series_id,country_code,country_name,year,Upper Secondary Educ. (% Pop.> 25yrs),Year Group
60104,SE.SEC.CUAT.UP.ZS,WSM,Samoa,2001,63.037628,2000 - 2005
60105,SE.SEC.CUAT.UP.ZS,TZA,Tanzania,2002,1.622880,2000 - 2005
60106,SE.SEC.CUAT.UP.ZS,UGA,Uganda,2002,6.453940,2000 - 2005
60107,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2002,10.357510,2000 - 2005
60108,SE.SEC.CUAT.UP.ZS,SLE,Sierra Leone,2003,4.066010,2000 - 2005
...,...,...,...,...,...,...
149288,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2017,12.264890,2016 - 2020
149290,SE.SEC.CUAT.UP.ZS,KEN,Kenya,2019,33.388390,2016 - 2020
149292,SE.SEC.CUAT.UP.ZS,ZMB,Zambia,2021,36.321079,2021 - 2023
149293,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2021,64.132111,2021 - 2023


In [28]:
WB_Educ_US_df_groups = WB_Educ_US_df.groupby(['year', 'country_name'])
WB_Educ_US_df_groups.head()

,series_id,country_code,country_name,year,Upper Secondary Educ. (% Pop.> 25yrs),Year Group
60104,SE.SEC.CUAT.UP.ZS,WSM,Samoa,2001,63.037628,2000 - 2005
60105,SE.SEC.CUAT.UP.ZS,TZA,Tanzania,2002,1.622880,2000 - 2005
60106,SE.SEC.CUAT.UP.ZS,UGA,Uganda,2002,6.453940,2000 - 2005
60107,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2002,10.357510,2000 - 2005
60108,SE.SEC.CUAT.UP.ZS,SLE,Sierra Leone,2003,4.066010,2000 - 2005
...,...,...,...,...,...,...
149288,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2017,12.264890,2016 - 2020
149290,SE.SEC.CUAT.UP.ZS,KEN,Kenya,2019,33.388390,2016 - 2020
149292,SE.SEC.CUAT.UP.ZS,ZMB,Zambia,2021,36.321079,2021 - 2023
149293,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2021,64.132111,2021 - 2023


In [29]:
avg_Educ_US_all = WB_Educ_US_df_groups[['Upper Secondary Educ. (% Pop.> 25yrs)']].mean()
avg_Educ_US_all

Upper Secondary Educ. (% Pop.> 25yrs)
year country_name                                       
2001 Samoa                                     63.037628
2002 Tanzania                                   1.622880
     Uganda                                     6.453940
     Zimbabwe                                  10.357510
2003 Sierra Leone                               4.066010
...                                                  ...
2022 Kenya                                     37.576099
     Nigeria                                   65.670242
     Rwanda                                    13.457430
     Somalia                                    9.807520
     Zambia                                    38.453289

[138 rows x 1 columns]

In [30]:
avg_Educ_US_yr = avg_Educ_US_all.reset_index()

In [31]:
avg_Educ_US_pivot1 = avg_Educ_US_yr.pivot(index='country_name', columns='year', values='Upper Secondary Educ. (% Pop.> 25yrs)')
avg_Educ_US_pivot1

year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,NaN,NaN,NaN,6.494953,NaN,NaN,NaN,NaN,7.719856,NaN,...,NaN,15.863530,17.436510,NaN,NaN,NaN,20.841118,NaN,22.049187,NaN
Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,42.949219,43.735836,44.229374,44.884315
Burkina Faso,NaN,NaN,NaN,NaN,NaN,4.007320,0.49510,NaN,NaN,NaN,...,NaN,2.805490,NaN,NaN,NaN,6.152450,7.821720,NaN,6.349400,NaN
Central African Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.612450,NaN,NaN,NaN
Chad,NaN,NaN,NaN,5.519980,NaN,NaN,NaN,NaN,5.104420,NaN,...,NaN,NaN,NaN,NaN,NaN,7.483197,4.556900,NaN,NaN,NaN
Djibouti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,15.623720,NaN,NaN,NaN,NaN,NaN
"Egypt, Arab Rep.",NaN,NaN,NaN,NaN,NaN,40.380260,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,67.156471,NaN,NaN,52.602589,51.787548,NaN
Ethiopia,NaN,NaN,NaN,NaN,NaN,NaN,7.02419,NaN,NaN,NaN,...,7.833063,NaN,NaN,6.92963,NaN,NaN,4.524770,NaN,NaN,NaN
Ghana,NaN,NaN,NaN,NaN,NaN,12.395874,NaN,NaN,NaN,20.563971,...,18.847855,NaN,22.084148,NaN,15.598560,NaN,NaN,NaN,NaN,NaN


In [32]:
avg_Educ_US_pivot1['Mean Mortality (2000 - 2023)'] = avg_Educ_US_pivot1.mean(axis=1)
avg_Educ_US_pivot1

year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Mean Mortality (2000 - 2023)
country_name,,,,,,,,,,,,,,,,,,,,,
Angola,NaN,NaN,NaN,6.494953,NaN,NaN,NaN,NaN,7.719856,NaN,...,15.863530,17.436510,NaN,NaN,NaN,20.841118,NaN,22.049187,NaN,14.672594
Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,42.949219,43.735836,44.229374,44.884315,43.949686
Burkina Faso,NaN,NaN,NaN,NaN,NaN,4.007320,0.49510,NaN,NaN,NaN,...,2.805490,NaN,NaN,NaN,6.152450,7.821720,NaN,6.349400,NaN,4.605247
Central African Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,5.612450,NaN,NaN,NaN,5.612450
Chad,NaN,NaN,NaN,5.519980,NaN,NaN,NaN,NaN,5.104420,NaN,...,NaN,NaN,NaN,NaN,7.483197,4.556900,NaN,NaN,NaN,5.666124
Djibouti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,15.623720,NaN,NaN,NaN,NaN,NaN,15.623720
"Egypt, Arab Rep.",NaN,NaN,NaN,NaN,NaN,40.380260,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,67.156471,NaN,NaN,52.602589,51.787548,NaN,52.981717
Ethiopia,NaN,NaN,NaN,NaN,NaN,NaN,7.02419,NaN,NaN,NaN,...,NaN,NaN,6.92963,NaN,NaN,4.524770,NaN,NaN,NaN,7.029541
Ghana,NaN,NaN,NaN,NaN,NaN,12.395874,NaN,NaN,NaN,20.563971,...,NaN,22.084148,NaN,15.598560,NaN,NaN,NaN,NaN,NaN,17.898082


In [33]:
# Split up our data into groups based upon 'country_name' and 'Year Group'
WB_Educ_US_groups2 = WB_Educ_US_df.groupby(['Year Group','country_name'])
WB_Educ_US_groups2.head()

,series_id,country_code,country_name,year,Upper Secondary Educ. (% Pop.> 25yrs),Year Group
60104,SE.SEC.CUAT.UP.ZS,WSM,Samoa,2001,63.037628,2000 - 2005
60105,SE.SEC.CUAT.UP.ZS,TZA,Tanzania,2002,1.622880,2000 - 2005
60106,SE.SEC.CUAT.UP.ZS,UGA,Uganda,2002,6.453940,2000 - 2005
60107,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2002,10.357510,2000 - 2005
60108,SE.SEC.CUAT.UP.ZS,SLE,Sierra Leone,2003,4.066010,2000 - 2005
...,...,...,...,...,...,...
149288,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2017,12.264890,2016 - 2020
149290,SE.SEC.CUAT.UP.ZS,KEN,Kenya,2019,33.388390,2016 - 2020
149292,SE.SEC.CUAT.UP.ZS,ZMB,Zambia,2021,36.321079,2021 - 2023
149293,SE.SEC.CUAT.UP.ZS,ZWE,Zimbabwe,2021,64.132111,2021 - 2023


In [34]:
avg_Educ_US_country = WB_Educ_US_groups2[['Upper Secondary Educ. (% Pop.> 25yrs)']].mean()
avg_Educ_US_country

Upper Secondary Educ. (% Pop.> 25yrs)
Year Group  country_name                                       
2000 - 2005 Angola                                     6.494953
            Chad                                       5.519980
            Malawi                                     7.952290
            Samoa                                     63.037628
            Sierra Leone                               8.097875
...                                                         ...
2021 - 2023 Rwanda                                    14.638091
            Somalia                                    9.807520
            Uganda                                    38.757221
            Zambia                                    37.387184
            Zimbabwe                                  64.132111

[82 rows x 1 columns]

In [35]:
avg_Educ_US_bin = avg_Educ_US_country.reset_index()

In [36]:
Educ_US_pivot2 = avg_Educ_US_bin.pivot(index='country_name', columns='Year Group', values='Upper Secondary Educ. (% Pop.> 25yrs)')
Educ_US_pivot2

Year Group,2000 - 2005,2006 - 2010,2011 - 2015,2016 - 2020,2021 - 2023
country_name,,,,,
Angola,6.494953,7.719856,15.201015,20.841118,22.049187
Botswana,NaN,NaN,NaN,43.342527,44.556845
Burkina Faso,NaN,2.251210,2.805490,6.987085,6.349400
Central African Republic,NaN,NaN,NaN,5.612450,NaN
Chad,5.519980,5.104420,NaN,6.020049,NaN
Djibouti,NaN,NaN,NaN,15.623720,NaN
"Egypt, Arab Rep.",NaN,40.380260,NaN,59.879530,51.787548
Ethiopia,NaN,7.024190,8.334557,5.727200,NaN
Ghana,NaN,16.479922,20.466002,15.598560,NaN


In [37]:
Educ_US_pivot2['Mean Mortality (2000 - 2023)'] = Educ_US_pivot2.mean(axis=1)
Educ_US_pivot2

Year Group,2000 - 2005,2006 - 2010,2011 - 2015,2016 - 2020,2021 - 2023,Mean Mortality (2000 - 2023)
country_name,,,,,,
Angola,6.494953,7.719856,15.201015,20.841118,22.049187,14.461226
Botswana,NaN,NaN,NaN,43.342527,44.556845,43.949686
Burkina Faso,NaN,2.251210,2.805490,6.987085,6.349400,4.598296
Central African Republic,NaN,NaN,NaN,5.612450,NaN,5.612450
Chad,5.519980,5.104420,NaN,6.020049,NaN,5.548150
Djibouti,NaN,NaN,NaN,15.623720,NaN,15.623720
"Egypt, Arab Rep.",NaN,40.380260,NaN,59.879530,51.787548,50.682446
Ethiopia,NaN,7.024190,8.334557,5.727200,NaN,7.028649
Ghana,NaN,16.479922,20.466002,15.598560,NaN,17.514828


## Educational Attainment : Masters (% Pop.> 25yrs)

In [38]:
WB_Educ_MS_df = WB_africadata_2000_df2.loc[WB_africadata_2000_df2['series_id'] == "SE.TER.CUAT.MS.ZS"]
WB_Educ_MS_df

,series_id,country_code,country_name,year,value,Year Group
72191,SE.TER.CUAT.MS.ZS,SLE,Sierra Leone,2003,0.210999,2000 - 2005
72192,SE.TER.CUAT.MS.ZS,AGO,Angola,2004,0.475335,2000 - 2005
72193,SE.TER.CUAT.MS.ZS,MWI,Malawi,2005,0.452600,2000 - 2005
72194,SE.TER.CUAT.MS.ZS,GHA,Ghana,2006,0.240368,2006 - 2010
72195,SE.TER.CUAT.MS.ZS,TZA,Tanzania,2008,0.128440,2006 - 2010
...,...,...,...,...,...,...
72281,SE.TER.CUAT.MS.ZS,ZMB,Zambia,2022,0.317543,2021 - 2023
149610,SE.TER.CUAT.MS.ZS,NGA,Nigeria,2006,1.642740,2006 - 2010
149611,SE.TER.CUAT.MS.ZS,TGO,Togo,2017,1.253972,2016 - 2020
149612,SE.TER.CUAT.MS.ZS,MRT,Mauritania,2019,1.938819,2016 - 2020


In [39]:
WB_Educ_MS_df = WB_Educ_MS_df.rename(columns = {'value': 'Masters Educ. (% Pop.> 25yrs)'})
WB_Educ_MS_df

,series_id,country_code,country_name,year,Masters Educ. (% Pop.> 25yrs),Year Group
72191,SE.TER.CUAT.MS.ZS,SLE,Sierra Leone,2003,0.210999,2000 - 2005
72192,SE.TER.CUAT.MS.ZS,AGO,Angola,2004,0.475335,2000 - 2005
72193,SE.TER.CUAT.MS.ZS,MWI,Malawi,2005,0.452600,2000 - 2005
72194,SE.TER.CUAT.MS.ZS,GHA,Ghana,2006,0.240368,2006 - 2010
72195,SE.TER.CUAT.MS.ZS,TZA,Tanzania,2008,0.128440,2006 - 2010
...,...,...,...,...,...,...
72281,SE.TER.CUAT.MS.ZS,ZMB,Zambia,2022,0.317543,2021 - 2023
149610,SE.TER.CUAT.MS.ZS,NGA,Nigeria,2006,1.642740,2006 - 2010
149611,SE.TER.CUAT.MS.ZS,TGO,Togo,2017,1.253972,2016 - 2020
149612,SE.TER.CUAT.MS.ZS,MRT,Mauritania,2019,1.938819,2016 - 2020


In [40]:
# Split up our data into groups based upon 'country_name' and 'Year Group'
WB_Educ_MS_groups = WB_Educ_MS_df.groupby(['year', 'country_name'])
WB_Educ_MS_groups.head()

,series_id,country_code,country_name,year,Masters Educ. (% Pop.> 25yrs),Year Group
72191,SE.TER.CUAT.MS.ZS,SLE,Sierra Leone,2003,0.210999,2000 - 2005
72192,SE.TER.CUAT.MS.ZS,AGO,Angola,2004,0.475335,2000 - 2005
72193,SE.TER.CUAT.MS.ZS,MWI,Malawi,2005,0.452600,2000 - 2005
72194,SE.TER.CUAT.MS.ZS,GHA,Ghana,2006,0.240368,2006 - 2010
72195,SE.TER.CUAT.MS.ZS,TZA,Tanzania,2008,0.128440,2006 - 2010
...,...,...,...,...,...,...
72281,SE.TER.CUAT.MS.ZS,ZMB,Zambia,2022,0.317543,2021 - 2023
149610,SE.TER.CUAT.MS.ZS,NGA,Nigeria,2006,1.642740,2006 - 2010
149611,SE.TER.CUAT.MS.ZS,TGO,Togo,2017,1.253972,2016 - 2020
149612,SE.TER.CUAT.MS.ZS,MRT,Mauritania,2019,1.938819,2016 - 2020


In [41]:
avg_Educ_MS_all = WB_Educ_MS_groups[['Masters Educ. (% Pop.> 25yrs)']].mean()
avg_Educ_MS_all

Masters Educ. (% Pop.> 25yrs)
year country_name                               
2003 Sierra Leone                       0.210999
2004 Angola                             0.475335
2005 Malawi                             0.452600
2006 Ghana                              0.240368
     Nigeria                            1.642740
...                                          ...
2021 Zimbabwe                           0.379013
2022 Botswana                           1.215302
     Nigeria                            2.852641
     Rwanda                             0.633470
     Zambia                             0.317543

[90 rows x 1 columns]

In [42]:
avg_Educ_MS_yr = avg_Educ_MS_all.reset_index()

In [43]:
Educ_MS_pivot1 = avg_Educ_MS_yr.pivot(index='country_name', columns='year', values='Masters Educ. (% Pop.> 25yrs)')
Educ_MS_pivot1

year,2003,2004,2005,2006,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country_name,,,,,,,,,,,,,,,,,,,
Angola,NaN,0.475335,NaN,NaN,NaN,3.022058,NaN,0.578822,NaN,NaN,0.257418,NaN,NaN,NaN,NaN,0.020671,NaN,0.000000,NaN
Botswana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.274187,0.979797,1.008132,1.215302
Burkina Faso,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,1.341010,1.180040,NaN,NaN,NaN
Chad,NaN,NaN,NaN,NaN,NaN,0.262690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.731943,NaN,NaN,NaN,NaN
Djibouti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
"Egypt, Arab Rep.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.692980,NaN,NaN,0.000000,0.000000,NaN
Ethiopia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.191011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ghana,NaN,NaN,NaN,0.240368,NaN,NaN,0.502748,NaN,NaN,0.510902,NaN,0.694553,NaN,0.988412,NaN,NaN,NaN,NaN,NaN
Kenya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.614439,NaN,1.000864,NaN


In [44]:
# Split up our data into groups based upon 'country_name' and 'Year Group'
WB_Educ_MS_groups2 = WB_Educ_MS_df.groupby(['Year Group','country_name'])
WB_Educ_MS_groups2.head()

,series_id,country_code,country_name,year,Masters Educ. (% Pop.> 25yrs),Year Group
72191,SE.TER.CUAT.MS.ZS,SLE,Sierra Leone,2003,0.210999,2000 - 2005
72192,SE.TER.CUAT.MS.ZS,AGO,Angola,2004,0.475335,2000 - 2005
72193,SE.TER.CUAT.MS.ZS,MWI,Malawi,2005,0.452600,2000 - 2005
72194,SE.TER.CUAT.MS.ZS,GHA,Ghana,2006,0.240368,2006 - 2010
72195,SE.TER.CUAT.MS.ZS,TZA,Tanzania,2008,0.128440,2006 - 2010
...,...,...,...,...,...,...
72281,SE.TER.CUAT.MS.ZS,ZMB,Zambia,2022,0.317543,2021 - 2023
149610,SE.TER.CUAT.MS.ZS,NGA,Nigeria,2006,1.642740,2006 - 2010
149611,SE.TER.CUAT.MS.ZS,TGO,Togo,2017,1.253972,2016 - 2020
149612,SE.TER.CUAT.MS.ZS,MRT,Mauritania,2019,1.938819,2016 - 2020


In [45]:
avg_Educ_MS_country = WB_Educ_MS_groups2[['Masters Educ. (% Pop.> 25yrs)']].mean()
avg_Educ_MS_country

Masters Educ. (% Pop.> 25yrs)
Year Group  country_name                                        
2000 - 2005 Angola                                      0.475335
            Malawi                                      0.452600
            Sierra Leone                                0.210999
2006 - 2010 Angola                                      3.022058
            Chad                                        0.262690
            Ghana                                       0.371558
            Liberia                                     0.515136
            Nigeria                                     1.642740
            Tanzania                                    0.115957
2011 - 2015 Angola                                      0.418120
            Burkina Faso                                0.000000
            Ethiopia                                    0.191011
            Ghana                                       0.602727
            Liberia                                     0.260976
            Madagascar                                  0.124399
            Malawi                                      0.634259
            Mauritania                                  1.237023
            Mozambique                                  2.188822
            Rwanda                                      0.223715
            Sao Tome and Principe                       0.000000
            Senegal                                     1.713577
            Sierra Leone                                0.625795
            Sudan                                       0.879369
            Tanzania                                    0.245774
            Togo                                        2.013687
            Uganda                                      0.360282
2016 - 2020 Angola                                      0.020671
            Botswana                                    1.126992
            Burkina Faso                                1.260525
            Chad                                        2.731943
            Djibouti                                    0.000000
            Egypt, Arab Rep.                            0.346490
            Ghana                                       0.988412
            Kenya                                       0.614439
            Lesotho                                     1.551489
            Liberia                                     1.888038
            Madagascar                                  1.517210
            Malawi                                      0.820643
            Mauritania                                  1.458544
            Mozambique                                  0.249100
            Nigeria                                     0.688488
            Rwanda                                      0.662580
            Senegal                                     2.189927
            Sierra Leone                                0.272927
            Tanzania                                    0.000000
            Togo                                        1.412945
            Tunisia                                     3.818086
            Uganda                                      0.000000
            Zambia                                      0.325500
            Zimbabwe                                    0.310750
2021 - 2023 Angola                                      0.000000
            Botswana                                    1.111717
            Egypt, Arab Rep.                            0.000000
            Kenya                                       1.000864
            Nigeria                                     2.852641
            Rwanda                                      0.650809
            Uganda                                      0.000000
            Zambia                                      0.398625
            Zimbabwe                                    0.379013

In [46]:
avg_Educ_MS_bin = avg_Educ_MS_country.reset_index()

In [47]:
Educ_MS_pivot2 = avg_Educ_MS_bin.pivot(index='country_name', columns='Year Group', values='Masters Educ. (% Pop.> 25yrs)')
Educ_MS_pivot2

Year Group,2000 - 2005,2006 - 2010,2011 - 2015,2016 - 2020,2021 - 2023
country_name,,,,,
Angola,0.475335,3.022058,0.418120,0.020671,0.000000
Botswana,NaN,NaN,NaN,1.126992,1.111717
Burkina Faso,NaN,NaN,0.000000,1.260525,NaN
Chad,NaN,0.262690,NaN,2.731943,NaN
Djibouti,NaN,NaN,NaN,0.000000,NaN
"Egypt, Arab Rep.",NaN,NaN,NaN,0.346490,0.000000
Ethiopia,NaN,NaN,0.191011,NaN,NaN
Ghana,NaN,0.371558,0.602727,0.988412,NaN
Kenya,NaN,NaN,NaN,0.614439,1.000864


## Africa Educational Attainment : Lower Secondary (% Pop.> 25yrs)

In [48]:
Educ_LS_pivot3 = avg_Educ_LS_bin.pivot(index='Year Group', columns='country_name', values='Lower Secondary Educ. (% Pop.> 25yrs)')
Educ_LS_pivot3

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Ghana,Kenya,...,Senegal,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe
Year Group,,,,,,,,,,,,,,,,,,,,,
2000 - 2005,22.165590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,17.709894,NaN,NaN,5.986470,NaN,NaN,11.518880,NaN,46.016670
2006 - 2010,10.561316,NaN,4.987165,NaN,10.044390,NaN,44.430729,7.215090,47.457388,30.992674,...,7.547190,NaN,NaN,NaN,12.754684,26.944210,38.207926,26.228886,NaN,NaN
2011 - 2015,26.587230,NaN,8.488350,NaN,NaN,NaN,NaN,15.085165,51.365938,NaN,...,11.801853,74.666176,NaN,27.881702,11.409920,36.165509,40.165585,24.025860,NaN,57.669699
2016 - 2020,38.469933,68.173122,13.336130,12.54875,11.051429,23.784451,62.780863,10.713265,46.434349,59.637825,...,14.030582,22.485196,NaN,NaN,7.260528,26.756779,43.517244,23.683174,53.084918,76.584721
2021 - 2023,39.977886,70.581100,11.453410,NaN,NaN,NaN,51.787548,NaN,NaN,68.872345,...,NaN,NaN,11.11157,NaN,NaN,NaN,NaN,39.458057,64.654213,89.623985


In [49]:
Educ_LS_pivot3['Mean Lower Secondary Educ. Attainment (2000 - 2023)'] = Educ_LS_pivot3.mean(axis=1)
Educ_LS_pivot3

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Ghana,Kenya,...,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe,Mean Lower Secondary Educ. Attainment (2000 - 2023)
Year Group,,,,,,,,,,,,,,,,,,,,,
2000 - 2005,22.165590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17.709894,NaN,NaN,5.986470,NaN,NaN,11.518880,NaN,46.016670,19.605817
2006 - 2010,10.561316,NaN,4.987165,NaN,10.044390,NaN,44.430729,7.215090,47.457388,30.992674,...,NaN,NaN,NaN,12.754684,26.944210,38.207926,26.228886,NaN,NaN,23.444013
2011 - 2015,26.587230,NaN,8.488350,NaN,NaN,NaN,NaN,15.085165,51.365938,NaN,...,74.666176,NaN,27.881702,11.409920,36.165509,40.165585,24.025860,NaN,57.669699,30.500509
2016 - 2020,38.469933,68.173122,13.336130,12.54875,11.051429,23.784451,62.780863,10.713265,46.434349,59.637825,...,22.485196,NaN,NaN,7.260528,26.756779,43.517244,23.683174,53.084918,76.584721,34.778185
2021 - 2023,39.977886,70.581100,11.453410,NaN,NaN,NaN,51.787548,NaN,NaN,68.872345,...,NaN,11.11157,NaN,NaN,NaN,NaN,39.458057,64.654213,89.623985,45.531446


## Africa Educational Attainment : Upper Secondary (% Pop.> 25yrs)

In [50]:
Educ_US_pivot3 = avg_Educ_US_bin.pivot(index='Year Group', columns='country_name', values='Upper Secondary Educ. (% Pop.> 25yrs)')
Educ_US_pivot3['Mean Upper Secondary Educ. Attainment (2000 - 2023)'] = Educ_US_pivot3.mean(axis=1)
Educ_US_pivot3

country_name,Angola,Botswana,Burkina Faso,Central African Republic,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Ghana,Kenya,...,Sierra Leone,Somalia,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe,Mean Upper Secondary Educ. Attainment (2000 - 2023)
Year Group,,,,,,,,,,,,,,,,,,,,,
2000 - 2005,6.494953,NaN,NaN,NaN,5.519980,NaN,NaN,NaN,NaN,NaN,...,8.097875,NaN,NaN,1.622880,NaN,NaN,6.453940,NaN,10.357510,13.692132
2006 - 2010,7.719856,NaN,2.251210,NaN,5.104420,NaN,40.380260,7.024190,16.479922,24.542450,...,NaN,NaN,NaN,1.840708,8.90192,NaN,9.720330,NaN,NaN,13.778175
2011 - 2015,15.201015,NaN,2.805490,NaN,NaN,NaN,NaN,8.334557,20.466002,NaN,...,51.155487,NaN,23.044186,4.334885,27.07158,NaN,9.869800,NaN,11.018940,19.708968
2016 - 2020,20.841118,43.342527,6.987085,5.61245,6.020049,15.62372,59.879530,5.727200,15.598560,31.381784,...,11.694002,NaN,NaN,3.541968,10.66748,28.599646,19.907260,30.928834,12.769595,18.797502
2021 - 2023,22.049187,44.556845,6.349400,NaN,NaN,NaN,51.787548,NaN,NaN,40.741879,...,NaN,9.80752,NaN,NaN,NaN,NaN,38.757221,37.387184,64.132111,32.823437


## Africa Educational Attainment : Masters (% Pop.> 25yrs)

In [51]:
Educ_MS_pivot3 = avg_Educ_MS_bin.pivot(index='Year Group', columns='country_name', values='Masters Educ. (% Pop.> 25yrs)')
Educ_MS_pivot3['Mean Masters Educ. Attainment (2000 - 2023)'] = Educ_MS_pivot3.mean(axis=1)
Educ_MS_pivot3

country_name,Angola,Botswana,Burkina Faso,Chad,Djibouti,"Egypt, Arab Rep.",Ethiopia,Ghana,Kenya,Lesotho,...,Senegal,Sierra Leone,Sudan,Tanzania,Togo,Tunisia,Uganda,Zambia,Zimbabwe,Mean Masters Educ. Attainment (2000 - 2023)
Year Group,,,,,,,,,,,,,,,,,,,,,
2000 - 2005,0.475335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.210999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.379644
2006 - 2010,3.022058,NaN,NaN,0.262690,NaN,NaN,NaN,0.371558,NaN,NaN,...,NaN,NaN,NaN,0.115957,NaN,NaN,NaN,NaN,NaN,0.988357
2011 - 2015,0.418120,NaN,0.000000,NaN,NaN,NaN,0.191011,0.602727,NaN,NaN,...,1.713577,0.625795,0.879369,0.245774,2.013687,NaN,0.360282,NaN,NaN,0.689384
2016 - 2020,0.020671,1.126992,1.260525,2.731943,0.0,0.34649,NaN,0.988412,0.614439,1.551489,...,2.189927,0.272927,NaN,0.000000,1.412945,3.818086,0.000000,0.325500,0.310750,1.010654
2021 - 2023,0.000000,1.111717,NaN,NaN,NaN,0.00000,NaN,NaN,1.000864,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.398625,0.379013,0.710408
